## Sunny Day Index Project
## DS 3000 - Spring 2023 - Northeastern University
## Authors: Brian Hayward, Jason Martinez, Noah Weinstein, Issac Schweiger
___

## Part 1: Data Processing and Analysis

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
%pip install plotly
import plotly.express as px
sns.set_style('darkgrid')

: 

In [ ]:
df = pd.read_csv('Data/Jan2017_to_Feb2023_data.csv', skiprows=3)
df.info()

: 

In [ ]:
df.isnull().sum()
# there are no null values, meaning that this dataset probably needs minimal processing

: 

In [ ]:
df.rename(columns={'weathercode (wmo code)': 'weathercode', 
                   'temperature_2m_max (°F)':'max_temperature', 
                   'temperature_2m_min (°F)':'min_temperature',
                   'temperature_2m_mean (°F)':'mean_temperature', 
                   'apparent_temperature_max (°F)':'max_feels_like', 
                   'apparent_temperature_min (°F)':'min_feels_like', 
                   'apparent_temperature_mean (°F)':'mean_feels_like', 
                   'sunrise (iso8601)':'sunrise_time', 
                   'sunset (iso8601)':'sunset_time', 
                   'shortwave_radiation_sum (MJ/m²)':'solar_radiation_sum', 
                   'rain_sum (inch)':'total_rainfall',
                   'snowfall_sum (inch)':'total_snowfall', 
                   'windspeed_10m_max (mp/h)':'max_windspeed', 
                   'windgusts_10m_max (mp/h)':'max_windgusts'}, inplace=True)
df.info()

: 

In [ ]:
df['time'] = pd.to_datetime(df['time'])
df['weathercode'] = df['weathercode'].astype('category')
df['sunrise_time'] = pd.to_datetime(df['sunrise_time'])
df['sunset_time'] = pd.to_datetime(df['sunset_time'])
df.info()

: 

In [ ]:
df.head()

#TODO EXTEND THE EDA WITH MORE DATA

: 

In [ ]:
fig = px.scatter(df,
              x = 'time',
              y = 'max_temperature',
              title = 'Max Temperature Over Time',
              color='max_temperature',
              color_continuous_scale='bluered',
              labels = {
                  'time': 'Time (UTC)', 
                  'max_temperature': 'Max Temperature (°F)'
                  })
fig.update_xaxes(rangeslider_visible=True)

: 

In [ ]:
fig = px.scatter(df,
              x = 'time',
              y = 'min_temperature',
              title = 'Minimum Temperature Over Time',
              color='min_temperature',
              color_continuous_scale='bluered',
              labels={
                  'time': 'Time (UTC)', 
                  'min_temperature': 'Minimum Temperature (°F)'}
              )
fig.update_xaxes(rangeslider_visible=True)

: 

In [ ]:
fig = px.scatter(df, 
              x='time',
              y='mean_temperature', 
              title='Mean Temperature Over Time',
              color='mean_temperature',
              color_continuous_scale='bluered',
              labels={
                  'time':'Time', 
                  'mean_temperature':'Mean Temperature (°F)'
                  })
fig.update_xaxes(rangeslider_visible=True)

: 

In [ ]:
fig = px.scatter(df, 
              x='time', 
              y='solar_radiation_sum', 
              title='Solar Radiation Over Time', 
              color='solar_radiation_sum',
              color_continuous_scale='sunset_r',
              labels={
                  'solar_radiation_sum':'Solar Radiation (MJ/m²)',
                  'time':'Time (UTC)'})
fig.update_xaxes(rangeslider_visible=True)

: 

In [ ]:
fig = px.scatter(df,
        x='time',
        y='total_rainfall',
        color='total_rainfall',
        color_continuous_scale='ice_r',
        title='Total Rainfall Over Time',
        labels={
                'total_rainfall':'Total Rainfall (in)', 
                'time':'Time (UTC)'
                })
fig.update_xaxes(rangeslider_visible=True)

: 

In [ ]:
fig = px.scatter(df,
              x='max_windspeed',
              y = 'total_rainfall',
              color='total_rainfall',
              )
fig.update_xaxes(rangeslider_visible=True)

: 

## Part 2: Splitting Data into Training and Testing Sets

In [ ]:
X   = df[['weathercode', 
           'max_temperature', 
           'min_temperature',
           'mean_temperature', 
           'max_feels_like', 
           'min_feels_like', 
           'mean_feels_like', 
           'solar_radiation_sum', 
           'total_snowfall', 
           'max_windspeed', 
           'max_windgusts']] #get the input features
y   = df['total_rainfall'] #get the target TODO MAKE IT PRECIPITATION WHICH IS EQUIVALENT TO THE SUM OF TOTAL_RAINFALL AND TOTAL_SNOWFALL

X_train, X_test, y_train, y_test = train_test_split(X,              #the input features
                                                    y,              #the label
                                                    test_size=0.3,  #set aside 30% of the data as the test set
                                                    random_state=7 #reproduce the results
                                                   )

: 

## Part 3: Building a Random Forest Regressor

In [ ]:
rf = RandomForestRegressor(random_state=7)
rf.fit(X_train, y_train)

: 

In [ ]:
#predict the labels for the test set
y_pred   = rf.predict(X_test)

print('The predicted precipitation is: {}'.format(y_pred))

: 

In [ ]:
mse = mean_squared_error(y_test, y_pred)

# Evaluate the Predictions
print('The mse of the model is: {}'.format(mse))

: 

In [ ]:
# Update target variable
y = df['total_rainfall'] + df['total_snowfall']

# Train the model
rf = RandomForestRegressor(random_state=7)
rf.fit(X, y)  # Train on the entire dataset


# Create a function to generate new data based on trends
def extrapolate_data(df, start_year, end_year):
    years_to_extrapolate = end_year - start_year
    df_ext = df.copy()
    
    for i in range(1, years_to_extrapolate + 1):
        df_temp = df.copy()
        df_temp['time'] = df_temp['time'] + pd.DateOffset(years=i)
        df_temp['sunrise_time'] = df_temp['sunrise_time'] + pd.DateOffset(years=i)
        df_temp['sunset_time'] = df_temp['sunset_time'] + pd.DateOffset(years=i)
        df_ext = pd.concat([df_ext, df_temp], ignore_index=True)
    
    return df_ext

# Extrapolate data up to 2028
df_2028 = extrapolate_data(df, 2023, 2028)

# Extract input features for the extrapolated data
X_2028 = df_2028[['weathercode', 
                  'max_temperature', 
                  'min_temperature',
                  'mean_temperature', 
                  'max_feels_like', 
                  'min_feels_like', 
                  'mean_feels_like', 
                  'solar_radiation_sum', 
                  'total_snowfall', 
                  'max_windspeed', 
                  'max_windgusts']]

# Make predictions for 2023-2028
y_pred_2028 = rf.predict(X_2028)

# Add predictions to the dataframe
df_2028['predicted_precipitation'] = y_pred_2028

# Save the predictions to a CSV file
df_2028.to_csv('Predicted_Weather_2023_to_2028.csv', index=False)

print("Predictions for 2023-2028 have been saved to 'Predicted_Weather_2023_to_2028.csv'")

: 

In [ ]:
import plotly.express as px  # Import the Plotly express module

# Plotting the actual vs predicted precipitation using seaborn and matplotlib
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.8)
plt.xlabel('Actual Precipitation')
plt.ylabel('Predicted Precipitation')
plt.title('Actual vs Predicted Precipitation - Random Forest Regressor')
plt.show()

# Plotting the distribution of prediction errors using seaborn
error = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(error, kde=True, bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Error - Random Forest Regressor')
plt.show()

: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert 'time' to pandas datetime format
df_2028['time'] = pd.to_datetime(df_2028['time'])

# Plot predicted precipitation over time
plt.figure(figsize=(12, 6))
sns.lineplot(x='time', y='predicted_precipitation', data=df_2028)
plt.xlabel('Date')
plt.ylabel('Predicted Precipitation')
plt.title('Predicted Precipitation Over Time - Random Forest Regressor')
plt.show()

# Calculate yearly average predicted precipitation
df_2028['year'] = df_2028['time'].dt.year
yearly_avg_pred_precip = df_2028.groupby('year')['predicted_precipitation'].mean()

# Plot yearly average predicted precipitation
plt.figure(figsize=(12, 6))
sns.barplot(x=yearly_avg_pred_precip.index, y=yearly_avg_pred_precip.values)
plt.xlabel('Year')
plt.ylabel('Yearly Average Predicted Precipitation')
plt.title('Yearly Average Predicted Precipitation (2023-2028) - Random Forest Regressor')
plt.show()

: 

## Part 4: Building a K-Nearest Neighbors Regressor

In [ ]:
k = 5  # Set the number of neighbors you want to consider
knn = KNeighborsRegressor(n_neighbors=k)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

: 

In [ ]:
# Plotting the actual vs predicted precipitation using seaborn and matplotlib
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.8)
plt.xlabel('Actual Precipitation')
plt.ylabel('Predicted Precipitation')
plt.title('Actual vs Predicted Precipitation')
plt.show()

# Plotting the distribution of prediction errors using seaborn
error = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(error, kde=True, bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.show()

: 

In [ ]:
# Convert 'time' to pandas datetime format
df_2028['time'] = pd.to_datetime(df_2028['time'])

# Plot predicted precipitation over time
plt.figure(figsize=(12, 6))
sns.lineplot(x='time', y='predicted_precipitation', data=df_2028)
plt.xlabel('Date')
plt.ylabel('Predicted Precipitation')
plt.title('Predicted Precipitation Over Time')
plt.show()

# Calculate yearly average predicted precipitation
df_2028['year'] = df_2028['time'].dt.year
yearly_avg_pred_precip = df_2028.groupby('year')['predicted_precipitation'].mean()

# Plot yearly average predicted precipitation
plt.figure(figsize=(12, 6))
sns.barplot(x=yearly_avg_pred_precip.index, y=yearly_avg_pred_precip.values)
plt.xlabel('Year')
plt.ylabel('Yearly Average Predicted Precipitation')
plt.title('Yearly Average Predicted Precipitation (2023-2028)')
plt.show()

: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm_model = SVR()
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

: 

In [ ]:
X_new = df_2028[['weathercode', 'max_temperature', 'min_temperature', 'mean_temperature', 'max_feels_like', 'min_feels_like', 'mean_feels_like', 'solar_radiation_sum', 'total_snowfall', 'max_windspeed', 'max_windgusts']]
X_new_scaled = scaler.transform(X_new)

predicted_precipitation = svm_model.predict(X_new_scaled)
df_2028['predicted_precipitation'] = predicted_precipitation

: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert 'time' to pandas datetime format
df_2028['time'] = pd.to_datetime(df_2028['time'])
df_2028 = df_2028.set_index('time')

# Calculate the average precipitation in a week for each month
weekly_avg_precip = df_2028.resample('W')['predicted_precipitation'].mean()

# Plot weekly average predicted precipitation
plt.figure(figsize=(15, 6))
sns.lineplot(x=weekly_avg_precip.index, y=weekly_avg_precip.values)
plt.xlabel('Date')
plt.ylabel('Weekly Average Predicted Precipitation')
plt.title('Weekly Average Predicted Precipitation Over Time')
plt.show()

# Calculate yearly average predicted precipitation
yearly_avg_pred_precip = df_2028.resample('Y')['predicted_precipitation'].mean()

# Plot yearly average predicted precipitation
plt.figure(figsize=(12, 6))
sns.barplot(x=yearly_avg_pred_precip.index.year, y=yearly_avg_pred_precip.values)
plt.xlabel('Year')
plt.ylabel('Yearly Average Predicted Precipitation')
plt.title('Yearly Average Predicted Precipitation (2023-2028)')
plt.show()

: 